<a href="https://colab.research.google.com/github/gina168651-dot/ai_0915/blob/main/1222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gradio as gr
def greet(name):
    return f"Hello {name}!"
demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9f10ecac00222bc0d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import gradio as gr

def calculator(num1, num2, operation):
    try:
        num1 = float(num1)
        num2 = float(num2)
    except ValueError:
        return "請輸入數字"

    if operation == "加法 (+)":
        result = num1 + num2
    elif operation == "減法 (-)":
        result = num1 - num2
    elif operation == "乘法 (*)":
        result = num1 * num2
    elif operation == "除法 (/)":
        if num2 == 0:
            return "除數不能為 0"
        result = num1 / num2
    else:
        return "未知運算"

    return f"結果：{result}"

with gr.Blocks() as demo:
    gr.Markdown("簡易計算機")

    # 橫向排列
    with gr.Row():
        num1 = gr.Number(label="數字 1", scale=2, min_width=100)
        num2 = gr.Number(label="數字 2", scale=2, min_width=100)
        operation = gr.Dropdown(
            choices=["加法 (+)", "減法 (-)", "乘法 (*)", "除法 (/)"],
            label="運算方式", scale=1, min_width=120
        )

    with gr.Row():
        btn = gr.Button("計算", variant="primary", scale=1)
        output = gr.Textbox(label="結果", scale=3)

    btn.click(fn=calculator, inputs=[num1, num2, operation], outputs=output)

demo.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://35fc8458c1e5359834.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
import io
import pandas as pd
import gradio as gr

# 解析上傳檔案（支援 .csv / .xlsx）
def parse_file(file_obj):
  if file_obj is None:
    return None, [], "請先上傳檔案"
  name = file_obj.name.lower()
  try:
    if name.endswith(".csv"):
      content = file_obj.read()
      try:
        df = pd.read_csv(io.BytesIO(content), encoding="utf-8")
      except UnicodeDecodeError:
        df = pd.read_csv(io.BytesIO(content), encoding="cp950")
    elif name.endswith(".xlsx"):
      df = pd.read_excel(file_obj)
    else:
      return None, [], "格式不支援，請上傳 .csv 或 .xlsx"

    cols = df.columns.tolist()
    # 修正：簡化載入成功訊息，移除列數和欄數
    msg = f"檔案 {name} 已成功載入。"
    return df, cols, msg
  except Exception as e:
    return None, [], f"讀檔錯誤：{e}"

# 分析主函式
def analyze(df, group_by_column, value_column, selected_stat):
  if df is None:
    return None, "尚未載入資料"
  if not group_by_column or not value_column or not selected_stat:
    return None, "請選擇分析欄位、計算欄位與統計方式"

  if group_by_column not in df.columns or value_column not in df.columns:
    return None, f"缺少必要欄位：{group_by_column} 或 {value_column}"

  try:
    if selected_stat == "最大值":
      res = df.groupby(group_by_column, dropna=False)[value_column].max()
    elif selected_stat == "最小值":
      res = df.groupby(group_by_column, dropna=False)[value_column].min()
    elif selected_stat == "平均":
      res = df.groupby(group_by_column, dropna=False)[value_column].mean()
    elif selected_stat == "合計":
      res = df.groupby(group_by_column, dropna=False)[value_column].sum()
    else:
      return None, "未知統計方式"

    res = res.to_frame(name=f"{selected_stat} - {value_column}").reset_index()
    return res, f"已完成：{group_by_column} 按 {value_column} 的 {selected_stat} 統計"
  except Exception as e:
    return None, f"分析錯誤：{e}"

with gr.Blocks(title="資料分析工具（Gradio 版）") as demo:
  gr.Markdown("## 📊 資料分析工具(Gradio)\n上傳 CSV / Excel,選擇群組欄位與統計方式，立即得到結果。")

  df_state = gr.State(None)

  with gr.Row():
    file_in = gr.File(label="上傳檔案（CSV 或 Excel）", file_types=[".csv", ".xlsx"], scale=2)
    load_msg = gr.Markdown("")

  with gr.Row():
    group_by_dropdown = gr.Dropdown(choices=[], label="選擇分析欄位（分組依據）", interactive=True, scale=1)
    value_dropdown = gr.Dropdown(choices=[], label="選擇計算欄位", interactive=True, scale=1)
    stat_dropdown = gr.Dropdown(choices=["最大值", "最小值", "平均", "合計"], label="選擇統計方式", value="合計", interactive=True, scale=1)
    run_btn = gr.Button("執行分析", variant="primary", scale=1)

  with gr.Row():
    result_df = gr.Dataframe(label="分析結果", interactive=False)

  # 事件：上傳檔案 → 解析、更新欄位清單
  def on_file_change(file_obj):
    df, cols, msg = parse_file(file_obj)
    if df is None:
      return None, gr.Dropdown(choices=[]), gr.Dropdown(choices=[]), msg

    if all(c in df.columns for c in ["北區", "中區", "南區"]):
      df["合計"] = df[["北區", "中區", "南區"]].sum(axis=1)
      cols.append("合計")

    return (
      df,
      gr.Dropdown(choices=cols, value=(cols[0] if cols else None)),
      gr.Dropdown(choices=cols, value="合計"),
      msg
    )

  file_in.change(
    on_file_change,
    inputs=[file_in],
    outputs=[df_state, group_by_dropdown, value_dropdown, load_msg]
  )

  # 事件：執行分析
  def on_run(df, group_by_column, value_column, selected_stat):
    res, msg = analyze(df, group_by_column, value_column, selected_stat)
    if res is None:
      return None, msg
    return res, msg

  run_btn.click(
    on_run,
    inputs=[df_state, group_by_dropdown, value_dropdown, stat_dropdown],
    outputs=[result_df, load_msg]
  )

demo.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c4690c8c9b9824925.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
!pip -q install gradio requests pandas


import requests
import pandas as pd
import gradio as gr
from urllib.parse import quote

# 取得 104 職缺列表（使用官方 list API，比解析 HTML 穩定）
def fetch_104_jobs(keyword: str, pages: int = 1):
  keyword = keyword.strip()
  if not keyword:
    return pd.DataFrame([{"訊息": "請輸入關鍵字"}])

  session = requests.Session()
  out_rows = []

  for page in range(1, pages + 1):
    url = "https://www.104.com.tw/jobs/search/list"
    params = {
      "ro": "0",        # 0:全部
      "kwop": "7",       # 關鍵字邏輯：AND
      "keyword": keyword,
      "order": "15",      # 依相關度排序
      "asc": "0",
      "page": str(page),
      "mode": "s",
      "jobsource": "joblist_search",
    }
    headers = {
      "User-Agent": "Mozilla/5.0",
      "Referer": f"https://www.104.com.tw/jobs/search/?keyword={quote(keyword)}&mode=s&order=15&jobsource=joblist_search",
      "Accept": "application/json, text/plain, */*",
    }

    try:
      r = session.get(url, params=params, headers=headers, timeout=15)
      r.raise_for_status()
      data = r.json()
      lst = data.get("data", {}).get("list", [])

      if not lst and page == 1:
        return pd.DataFrame([{"訊息": "找不到相關職缺，換個關鍵字試試？"}])

      for it in lst:
        # 常見欄位：職稱、公司、地點、薪資、產業、刊登日、工作連結
        job_name  = it.get("jobName", "")
        company  = it.get("custName", "")
        area    = it.get("jobAddrNoDesc", "") # 縣市+區
        salary   = it.get("salaryDesc", "")
        industry  = it.get("industryDesc", "")
        date    = it.get("appearDate", "")
        job_no   = it.get("jobNo", "")
        job_link  = f"https://www.104.com.tw/job/{job_no}" if job_no else ""

        out_rows.append({
          "職務名稱": job_name,
          "公司名稱": company,
          "地點": area,
          "薪資": salary,
          "產業": industry,
          "刊登日": date,
          "連結": job_link,
        })

    except Exception as e:
      return pd.DataFrame([{"錯誤": f"{type(e).__name__}: {e}"}])

  df = pd.DataFrame(out_rows)
  if df.empty:
    return pd.DataFrame([{"訊息": "查無結果"}])

  # 讓連結可點（Gradio DataFrame不支援HTML，保留純文字連結）
  return df


# --- Gradio 介面 ---
with gr.Blocks(title="104 職缺查詢（Colab 版）") as demo:
  gr.Markdown("## 104 職缺查詢\n在下方輸入技能/職稱關鍵字（例如：**Python**、**資料分析**）。")
  with gr.Row():
    kw = gr.Textbox(label="關鍵字", value="Python")
    pages = gr.Slider(1, 5, value=1, step=1, label="頁數（每頁約 20 筆）")
    btn = gr.Button("搜尋", variant="primary")

  table = gr.Dataframe(headers=["職務名稱","公司名稱","地點","薪資","產業","刊登日","連結"], wrap=True)

  btn.click(fn=fetch_104_jobs, inputs=[kw, pages], outputs=table)

demo.launch()






It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://37bdd7c6640d9f2255.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
